In [11]:
import sys
sys.path.append("..")
import xml_converter as xmlc
from xml.etree import cElementTree as ElementTree
from lxml import etree as letree
import os
import json
from glob import glob


In [2]:
ETL_SAMPLES_PATH = '/v/region/na/appl/itlnc/s8/data/uat/silent8/name-screening/etl_samples/'
XML_PATH = '/v/region/na/appl/itlnc/s8/data/uat/datamart_extracts/'

In [3]:
DATASET_NAME = 'ISG_Weekly' # 'ISG_Daily' # 'WM_Party' # 'ISG_Weekly 
DATASET_TYPE =  'Party' # 'Address' # Party # Account
DATASET = DATASET_NAME + '_' + DATASET_TYPE

In [4]:
sampels = {
    'ISG_Daily_Party': ETL_SAMPLES_PATH + 'supplemental_info/ISG_Daily_Party/',
    'ISG_Weekly_Party': ETL_SAMPLES_PATH + 'supplemental_info/ISG_Weekly_Party/',
    'ISG_Daily_Account': ETL_SAMPLES_PATH + 'supplemental_info/ISG_Daily_Account/',
    'ISG_Weekly_Account': ETL_SAMPLES_PATH + 'supplemental_info/ISG_Weekly_Account/',
    'WM_Party': ETL_SAMPLES_PATH + 'supplemental_info/WM_Party/',
    'WM_Address': ETL_SAMPLES_PATH + 'supplemental_info/WM_Address/'
}
xmls = {
    'ISG_Daily_Party': XML_PATH + 'ISG/Daily/Party/XMLs/Alert_Wise_XML/',
    'ISG_Weekly_Party': XML_PATH + 'ISG/Weekly/Party/XMLs/Alert_Wise_XML/',
    'ISG_Daily_Account': XML_PATH + 'ISG/Daily/Account/XMLs/Alert_Wise_XML/',
    'ISG_Weekly_Account': XML_PATH + 'ISG/Weekly/Account/XMLs/Alert_Wise_XML/',
    'WM_Party': XML_PATH + 'WM/Party/XMLs/Alert_Wise_XML/',
    'WM_Address': XML_PATH + 'WM/Address/XMLs/Alert_Wise_XML/'
}

In [5]:
samples_supplemental_info = glob(sampels[DATASET] + '*.json')
all_xmls = glob(xmls[DATASET] + '*.xml')

In [ ]:
with open(all_xmls[0], 'rb') as file:
    xml = file.read().replace(b'</inquiryDisposition></alert>', b'')
    root = ElementTree.XML(xml)
    xmldict = xmlc.XmlDictConfig(root)

xmldict

In [ ]:
JSON_DIR = f'{ETL_SAMPLES_PATH}/complete_info/{DATASET}'
%mkdir -p {JSON_DIR}

In [16]:
sample = "sample/supplementalInfo.json"
ref_xml = "sample/sourceXml.xml"


In [20]:
def create_template(sample, ref_xml):
    template = {}
    with open(sample, 'r') as suppl_file, open(ref_xml, 'rb') as xml_file:
            xml = xml_file.read() # .replace(b'</inquiryDisposition></alert>', b'')
            suppl = suppl_file.read()

        # root = ElementTree.XML(xml)
    parser = letree.XMLParser(recover=True)
    root = letree.fromstring(xml, parser=parser)
    xmldict = xmlc.XmlDictConfig(root)
    
    template['supplementalInfo'] = json.loads(suppl)['supplementalInfo']
    template['alert'] = xmldict

    template['alert']['matchRecords'] = template['alert']['matchRecords']['matchRecord']

    if not isinstance(template['alert']['matchRecords'], list):
        template['alert']['matchRecords'] = [template['alert']['matchRecords']]
    for match in xmldict['matchRecords']:
        import pdb; pdb.set_trace()
        if not isinstance(match['stopDescriptors']['stopDescriptor'], list):
            match['stopDescriptors'] = [match['stopDescriptors']['stopDescriptor']]
        else:
            match['stopDescriptors'] = [item['stopDescriptor'] for item in match['stopDescriptors']]


    template['alert']['inputRecordHist'] = template['alert']['inputRecordHist']['inputRecord']

    if not isinstance(template['alert']['inputRecordHist'], list):
        template['alert']['inputRecordHist'] = [template['alert']['inputRecordHist']]

    for item in template['alert']['inputRecordHist']:
        if not isinstance(item['field'], list):
            item['field'] = [item['field']]
            
    return template



In [9]:
for sample in tqdm(samples_supplemental_info):
    alert_no = os.path.split(sample)[1][:-4]
    
    try:
        ref_xml = next(filter(lambda x: alert_no in x, all_xmls))
    except StopIteration:
        continue
    
    template = create_template(sample, ref_xml)
    # print(template)

    with open(JSON_DIR + f'/{alert_no}.json', 'w') as file:
        file.write(json.dumps(template, indent=2))

NameError: name 'tqdm' is not defined

In [21]:
template = create_template(sample, ref_xml)

> /tmp/ipykernel_201076/438320330.py(21)create_template()
     19     for match in xmldict['matchRecords']:
     20         import pdb; pdb.set_trace()
---> 21         if not isinstance(match['stopDescriptors']['stopDescriptor'], list):
     22             match['stopDescriptors'] = [match['stopDescriptors']['stopDescriptor']]
     23         else:



ipdb>  n


> /tmp/ipykernel_201076/438320330.py(24)create_template()
     22             match['stopDescriptors'] = [match['stopDescriptors']['stopDescriptor']]
     23         else:
---> 24             match['stopDescriptors'] = [item['stopDescriptor'] for item in match['stopDescriptors']]
     25 
     26 



ipdb>  match['stopDescriptors']['stopDescriptor']


[{'name': 'Agent Smith', 'totalMatchScore': '0.9090909090909091', 'stopDescriptorDetail': {'inputToken': 'XXX', 'inputSynonym': 'null', 'sdToken': 'Smith', 'matchScore': '0.75'}}, {'name': 'Agent Smith', 'totalMatchScore': '0.9090909090909091', 'stopDescriptorDetail': {'inputToken': 'XXX', 'inputSynonym': 'null', 'sdToken': 'Smith', 'matchScore': '0.75'}}]


ipdb>  qui


*** NameError: name 'qui' is not defined


ipdb>  c


TypeError: string indices must be integers

In [10]:

with open("sample/alert.json", 'w') as file:
    file.write(json.dumps(template, indent=2))